In [1]:
import pandas as pd
import os 
import h3 as h3
import matplotlib.pyplot as plt
import plotly
import folium
import webbrowser
from folium import Map

from IPython.core.display import display, HTML
import json
from pandas.io.json import json_normalize
import numpy as np

import statistics

from geojson.feature import *
# from area import area

import copy
from folium import Map, Marker, GeoJson
from folium.plugins import MarkerCluster
import branca.colormap as cm
from branca.colormap import linear

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image, display
from IPython.utils.text import columnize
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df = pd.read_csv('~/Downloads/Book3_England.csv')    # english parliamentary constituencies
ori = pd.read_csv('~/Downloads/Book3_England.csv')    # english parliamentary constituencies

print(df.head())

               constituency       lat      lng  population  no_of_foodbanks  \
0                 Aldershot  51.28540 -0.77700      105601                4   
1       Aldridge-Brownhills  52.61431 -1.92815       77476                1   
2  Altrincham and Sale West  53.39000 -2.36570      101861                3   
3              Amber Valley  53.03994 -1.40000       90620                1   
4   Arundel and South Downs  50.92779 -0.45467      101579                6   

   index_of_multiple_deprivation  Unnamed: 6  Unnamed: 7  Unnamed: 8  \
0                          15.00         NaN         NaN         NaN   
1                          18.72         NaN         NaN         NaN   
2                          10.39         NaN         NaN         NaN   
3                          20.90         NaN         NaN         NaN   
4                          11.21         NaN         NaN         NaN   

   Unnamed: 9  Unnamed: 10  Unnamed: 11  
0         NaN          NaN          NaN  
1       

In [3]:


def counts_by_hexagon(df, resolution):

    '''Use h3.geo_to_h3 to index each data point into the spatial index of the specified resolution.
      Use h3.h3_to_geo_boundary to obtain the geometries of these hexagons'''

    df = df[["lat","lng"]]

    df["hex_id"] = df.apply(lambda row: h3.geo_to_h3(row["lat"], row["lng"], resolution), axis = 1)

    df_aggreg = df
    df_aggreg['index_of_multiple_deprivation'] = ori['index_of_multiple_deprivation']
    df_aggreg['constituency'] = ori['constituency']

    df_aggreg.columns = ["lat", "lng", "hex_id", "index_of_multiple_deprivation", "constituency"]

    df_aggreg["geometry"] =  df_aggreg.hex_id.apply(lambda x: 
                                                           {    "type" : "Polygon",
                                                                 "coordinates": 
                                                                [h3.h3_to_geo_boundary(x,geo_json=True)]
                                                            }
                                                        )

    return df_aggreg


def hexagons_dataframe_to_geojson(df_hex, file_output = None):
    """
    Produce the GeoJSON for a dataframe that has a geometry column in geojson 
    format already, along with the columns hex_id and value

    Ex counts_by_hexagon(data)
    """    
    list_features = []

    for i,row in df_hex.iterrows():
        feature = Feature(geometry = row["geometry"] , id=row["hex_id"], properties = {"index_of_multiple_deprivation" : row["index_of_multiple_deprivation"]})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)

    geojson_result = json.dumps(feat_collection)

    #optionally write to file
    if file_output is not None:
        with open(file_output,"w") as f:
            json.dump(feat_collection,f)

    return geojson_result

def choropleth_map(df_aggreg, border_color = 'black', fill_opacity = 0.7, initial_map = None, with_legend = False,
               kind = "linear"):

    """
    Creates choropleth maps given the aggregated data.
    """    
    #colormap
    min_value = df_aggreg["index_of_multiple_deprivation"].min()
    max_value = df_aggreg["index_of_multiple_deprivation"].max()
    m = round ((min_value + max_value ) / 2 , 0)

    #take resolution from the first row
    res = h3.h3_get_resolution(df_aggreg.loc[0,'hex_id'])

    if initial_map is None:
        initial_map = Map(location= [51.53, 0.11], zoom_start=7, tiles="cartodbpositron", 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>' 
            )


    #the colormap 
    #color names accepted https://github.com/python-visualization/branca/blob/master/branca/_cnames.json
    if kind == "linear":
        custom_cm = cm.LinearColormap(['green','yellow','red'], vmin=min_value, vmax=max_value)
    elif kind == "outlier":
        #for outliers, values would be -11,0,1
        custom_cm = cm.LinearColormap(['blue','white','red'], vmin=min_value, vmax=max_value)
    elif kind == "filled_nulls":
        custom_cm = cm.LinearColormap(['sienna','green','yellow','red'], 
                                      index=[0,min_value,m,max_value],vmin=min_value,vmax=max_value)


    #create geojson data from dataframe
    geojson_data = hexagons_dataframe_to_geojson(df_hex = df_aggreg)

    #plot on map
    name_layer = "Choropleth " + str(res)
    if kind != "linear":
        name_layer = name_layer + kind

    GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': custom_cm(feature['properties']['index_of_multiple_deprivation']),
            'color': border_color,
            'weight': 1,
            'fillOpacity': fill_opacity 
        }, 
        name = name_layer
    ).add_to(initial_map)
    #add legend (not recommended if multiple layers)
    if with_legend == True:
        custom_cm.add_to(initial_map)



    return initial_map

def plot_scatter(df, metric_col, x='lng', y='lat', marker='.', alpha=1, figsize=(16,12), colormap='viridis'):
    """
    Scatter plot function for h3 indexed objects
    """    
    df.plot.scatter(x=x, y=y, c=metric_col, title=metric_col
                    , edgecolors='none', colormap=colormap, marker=marker, alpha=alpha, figsize=figsize);
    plt.xticks([], []); plt.yticks([], [])


In [4]:
# Counts how many points are within the hex
myres = 6
df_aggreg = counts_by_hexagon(df = df, resolution = myres)      #Use 4 to tile the whole of uk in an aesthetically pleasing way
df_aggreg.sort_values(by = "index_of_multiple_deprivation", ascending = False, inplace = True)

# Creates a map 
hexmap = choropleth_map(df_aggreg = df_aggreg, with_legend = True)
# hexmap.save('downloads')
from folium import plugins
plugins.Fullscreen(position='topright').add_to(hexmap)
draw = plugins.Draw(export=True)

In [5]:
hexmap

In [6]:
df_aggreg

,lat,lng,hex_id,index_of_multiple_deprivation,constituency,geometry
264,53.446300,-2.948430,861951077ffffff,55.68,"Liverpool, Walton","{'type': 'Polygon', 'coordinates': [((-2.97132..."
33,52.489170,-1.813600,86195c04fffffff,49.34,"Birmingham, Hodge Hill","{'type': 'Polygon', 'coordinates': [((-1.80823..."
43,53.798300,-3.033430,861951897ffffff,48.36,Blackpool South,"{'type': 'Polygon', 'coordinates': [((-3.03028..."
41,53.516020,-2.233520,8619424dfffffff,46.85,Blackley and Broughton,"{'type': 'Polygon', 'coordinates': [((-2.19869..."
31,52.525010,-1.837450,86195c047ffffff,46.70,"Birmingham, Erdington","{'type': 'Polygon', 'coordinates': [((-1.84067..."
...,...,...,...,...,...,...
378,52.879197,-1.082215,861943417ffffff,7.16,Rushcliffe,"{'type': 'Polygon', 'coordinates': [((-1.06740..."
532,53.958937,-1.058548,8619429afffffff,7.01,York Outer,"{'type': 'Polygon', 'coordinates': [((-1.09022..."
101,51.680530,-0.627790,86195da9fffffff,6.95,Chesham and Amersham,"{'type': 'Polygon', 'coordinates': [((-0.61372..."
314,51.271030,-0.992980,861959907ffffff,6.19,North East Hampshire,"{'type': 'Polygon', 'coordinates': [((-1.02594..."
